In [1]:
#!pip install transformers
import numpy as np
import pandas as pd
from transformers import BertModel, BertTokenizer, BertConfig
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch
import torch.nn as nn
from sklearn import metrics
from torch import cuda
import ast

In [2]:
train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')
test_df = pd.read_csv('test.csv')
#set(val_df['author'].tolist()+train_df['author'].tolist())

In [3]:
#https://colab.research.google.com/github/abhimishra91/transformers-tutorials/blob/master/transformers_multi_label_classification.ipynb#scrollTo=7KnNeQx6SI78
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 5
LEARNING_RATE = 8e-05
NUM_CLASSES = len(ast.literal_eval(train_df.loc[0,'label']))
device = 'cuda' if cuda.is_available() else 'cpu'

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
device

'cpu'

In [4]:
def get_y(x):
    return np.asarray(ast.literal_eval(x), dtype=int)

In [5]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        y = self.data['label'].apply(get_y)
        y1 = []
        for i in y:
                y1.append(i)
        y1 = np.asarray(y1)
        self.targets =np.asarray(y1)
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        inputs = self.tokenizer.encode_plus(
            self.text[index],
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long).to(device, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long).to(device, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long).to(device, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float).to(device, dtype=torch.float)
        }


training_set = CustomDataset(train_df.reset_index(drop=True), tokenizer, MAX_LEN)
val_set = CustomDataset(val_df.reset_index(drop=True), tokenizer, MAX_LEN)
test_set = CustomDataset(test_df.reset_index(drop=True), tokenizer, MAX_LEN)
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
               'shuffle': True,
               'num_workers': 0
               }

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **test_params)
test_loader = DataLoader(test_set, **test_params)

In [6]:
def create_model():
    model = BertModel.from_pretrained(model_name)

    for param in model.parameters():
        param.requires_grad = False

    # Replace the classification layer with a linear layer
    model.classifier = torch.nn.Sequential(
        torch.nn.Dropout(0.1),
        torch.nn.Linear(model.config.hidden_size, NUM_CLASSES)
    )

    # Unfreeze the weights of the last 4 layers
    for param in model.encoder.layer[-4:].parameters():
        param.requires_grad = True

    model.to(device)
    return model


In [16]:
class BertForClassification(nn.Module):
    def __init__(self, config):
        super(BertForClassification, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, NUM_CLASSES)

    def forward(self, input_ids, token_type_ids, attention_mask):
        outputs = self.bert(input_ids, token_type_ids, attention_mask)
        pooled_output = self.dropout(outputs.pooler_output)
        logits = self.classifier(pooled_output)
        return logits


model = BertForClassification(BertConfig())
# Fine-tune the model on your downstream task
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
for param in model.bert.encoder.layer.parameters():
        param.requires_grad = False
    # Unfreeze the weights of the last 4 layers
for param in model.bert.encoder.layer[-4:].parameters():
    param.requires_grad = True

    model.to(device)
model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [17]:
#https://stackoverflow.com/questions/71998978/early-stopping-in-pytorch
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [18]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [19]:
def val(epoch, val_loader: DataLoader, model: BertModel):
    with torch.no_grad():
        model.eval()
        fin_targets = []
        fin_outputs = []
        for _, data in enumerate(val_loader, 0):
            outputs = model(data['ids'],
                            attention_mask=data['mask'],
                            token_type_ids=data['token_type_ids'])
            preds = torch.argmax(outputs, dim=1)
            labels = torch.argmax(data['targets'], dim=1)
            fin_targets.extend(labels)
            fin_outputs.extend(preds)
            loss = loss_fn(outputs, data['targets'])
        acc = (torch.tensor(fin_targets) == torch.tensor(fin_outputs)).float().mean().item()
        print(f"Epoch {epoch + 1}, Val accuracy = {acc}")
        return loss


def train(epoch, training_loader: DataLoader, val_loader: DataLoader, model: BertModel):
    early_stopper = EarlyStopper(patience=3)
    for epoch in range(epoch):
        model.train()
        for _, data in enumerate(training_loader, 0):
            outputs = model(data['ids'],
                            attention_mask=data['mask'],
                            token_type_ids=data['token_type_ids'])
            loss = loss_fn(outputs, data['targets'])
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if _ % 100 == 0:
                print(f"Epoch {epoch + 1}, Loss= {loss.item()}")
        # Evaluate the model on the dev set
        validation_loss = val(epoch, val_loader, model)
        if early_stopper.early_stop(validation_loss):
            break


def test(test_loader, model: BertModel):
    # Use the fine-tuned model to make predictions on new data
    with torch.no_grad():
        fin_targets = []
    fin_outputs = []
    for _, data in enumerate(test_loader, 0):
        outputs = model(data['ids'],
                        attention_mask=data['mask'],
                        token_type_ids=data['token_type_ids'])
        preds = torch.argmax(outputs, dim=1)
        labels = torch.argmax(data['targets'], dim=1)
        fin_targets.extend(labels.cpu().detach().numpy().tolist())
        fin_outputs.extend(preds.cpu().detach().numpy().tolist())
    test_df['truth'] = fin_targets
    test_df['pred'] = fin_outputs
    acc = (torch.tensor(fin_targets) == torch.tensor(fin_outputs)).float().mean().item()
    print(f"Test accuracy = {acc}")


train(EPOCHS, training_loader, val_loader, model)
test(test_loader, model)
test_df[['text', 'truth', 'pred']]

/opt/homebrew/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch 1, Loss= 0.6997514963150024


KeyboardInterrupt: 